[`Simple TFX Pipeline Tutorial using Penguin dataset`](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)

02/07/2026  
Alan MH Beem
(I didn't write the tutorial, just this file)

<!-- [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AlanBeem/Penguin_tfx/blob/main/main.ipynb) -->
<!-- Button disabled due to lack of functionality in Colab -->

Due to a [known issue](https://github.com/tensorflow/tfx/issues/7748) with `tfx`, a roundabout installation path is taken, here, using MacOS with Apple Silicon running as x86 in Rosetta Terminal

Preliminaries (imports, variables)

In [1]:
import os
from absl import logging
import urllib.request
import tempfile

In [ ]:
from tfx.orchestration.local.local_dag_runner import LocalDagRunner
from tfx.orchestration import pipeline

from tfx.components import (
    CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator,
    Transform, Trainer, Evaluator, Pusher
)

Setup on macOS for local run of this tutorial

In [ ]:
# #
# # WORKING ROUTE TO LOCAL INSTALL OF TFX ON MACOS (but SLOW 64+minute import)
# #
# # (1) set up rosetta terminal on macOS
# # (2) start a rosetta terminal session
# # (3) create a conda environment using a specific subdirectory
# # (4) activate new environment
# # (5) change conda config to stay in subdirectory in new env.
# # (6) install with update: setuptools, wheel
# # (7) install 1.16.0 of tfx

# # (1)
# softwareupdate --install-rosetta --agree-to-license

# # (2)
# arch -x86_64 zsh

# # (3)
# CONDA_SUBDIR=osx-64 conda create -n tfx116_x86 python=3.10 -y

# # (4)
# conda activate tfx116_x86

# # (5)
# conda config --env --set subdir osx-64

# # (6)
# python -m pip install -U pip setuptools wheel

# # (7)
# python -m pip install "tfx==1.16.0"


# # a useful command to remove unnecessary cached copies of packages (4.4 GB
# # here, accumulated over about 2 years of heavy undergraduate Python)
# conda clean --all

# # switch the kernel of the notebook to this new terminal to use tfx

Back to the tutorial

In [ ]:
# import tensorflow as tf
# print(f'TensorFlow version: {tf.__version__}')
# from tfx import v1 as tfx
# print(f'TFX version: {tfx.__version__}')

In [ ]:
PIPELINE_NAME = "penguin-simple"

# for artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# path to an SQLite DB file to use as an MLMD storage.
PIPELINE_METADATA = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# the location to which created models will be exported.
PIPELINE_SERVING = os.path.join('serving_model', PIPELINE_NAME)

logging.set_verbosity(logging.INFO)  # Set default logging level.

Example data

In [ ]:
DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

In [ ]:
head {_data_filepath}